In [74]:
# Append web folder to system path
import sys
sys.path.append('../web')

In [75]:
from db_config import Base
from Database import Station, Availability, Weather, WeatherPredictive
from sqlalchemy import create_engine, func, Column, String, Integer, Double, Boolean, MetaData
from sqlalchemy.orm import sessionmaker
import json
import sys
import requests
from datetime import datetime, timedelta

In [76]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

In [101]:
# Get the db_info
with open('../dbinfo.json') as f:
    db_info = json.load(f)


USER = db_info['dbConnection']['USER']
PASSWORD = db_info['dbConnection']['PASSWORD']
URI = db_info['dbConnection']['URI']
PORT = db_info['dbConnection']['PORT']
DB = db_info['dbConnection']['DB']
WEATHER_API_KEY = db_info['weatherKey']
FORECAST_URI = 'https://api.openweathermap.org/data/2.5/forecast'


# Create a new session
engine = create_engine(
    'mysql+pymysql://{}:{}@localhost:{}/{}'.format(USER, PASSWORD, PORT, DB), echo=True)
Base.metadata.create_all(bind=engine)
Session = sessionmaker(bind=engine)
session = Session()
print("connected")

2024-04-04 11:12:37,522 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2024-04-04 11:12:37,524 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-04 11:12:37,548 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2024-04-04 11:12:37,549 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-04 11:12:37,558 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2024-04-04 11:12:37,560 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-04 11:12:37,578 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-04 11:12:37,585 INFO sqlalchemy.engine.Engine DESCRIBE `db_on_your_bike`.`stations`
2024-04-04 11:12:37,586 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-04 11:12:37,595 INFO sqlalchemy.engine.Engine DESCRIBE `db_on_your_bike`.`availability`
2024-04-04 11:12:37,596 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-04 11:12:37,608 INFO sqlalchemy.engine.Engine DESCRIBE `db_on_your_bike`.`weather`
2024-04-04 11:12:37,608 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-04 11:12:3

In [102]:
station_id = 1
midnight = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)

station_information = session.query(Availability).filter(
Availability.time_updated > midnight, Availability.station_id == station_id).all()

station_data = pd.DataFrame([row.__dict__ for row in station_information])
station_data = station_data.groupby(station_data['time_updated'].dt.floor('H')).agg({
'available_bikes': 'mean',  # Example: Calculate mean temperature for each hour
'bike_stands': 'mean'      # Example: Calculate mean humidity for each hour
}).reset_index()

2024-04-04 11:12:39,860 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-04 11:12:39,899 INFO sqlalchemy.engine.Engine SELECT availability.station_id AS availability_station_id, availability.time_updated AS availability_time_updated, availability.bike_stands AS availability_bike_stands, availability.available_bikes AS availability_available_bikes, availability.available_bike_stands AS availability_available_bike_stands, availability.status AS availability_status 
FROM availability 
WHERE availability.time_updated > %(time_updated_1)s AND availability.station_id = %(station_id_1)s
2024-04-04 11:12:39,903 INFO sqlalchemy.engine.Engine [generated in 0.00405s] {'time_updated_1': datetime.datetime(2024, 4, 4, 0, 0), 'station_id_1': 1}


/var/folders/6q/d_h4k1216h90crzgkghkzc9r0000gp/T/ipykernel_1026/490537819.py:8: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  station_data = station_data.groupby(station_data['time_updated'].dt.floor('H')).agg({


In [103]:
station_data['hour'] = station_data['time_updated'].dt.hour
station_data = station_data[['hour', 'available_bikes', 'bike_stands']]


In [104]:
weather_predictive = session.query(WeatherPredictive).all()
weather_predictive_df = pd.DataFrame(
        [row.__dict__ for row in weather_predictive])
weather_predictive_df['type'] = weather_predictive_df['weather_type']
weather_predictive_df = weather_predictive_df[[
        'time_updated', 'temperature', 'wind_speed', 'humidity', 'type']]



2024-04-04 11:12:45,242 INFO sqlalchemy.engine.Engine SELECT weather_predictive.forecast_time AS weather_predictive_forecast_time, weather_predictive.type AS weather_predictive_type, weather_predictive.description AS weather_predictive_description, weather_predictive.temperature AS weather_predictive_temperature, weather_predictive.feels_like AS weather_predictive_feels_like, weather_predictive.min_temp AS weather_predictive_min_temp, weather_predictive.max_temp AS weather_predictive_max_temp, weather_predictive.rain_3h AS weather_predictive_rain_3h, weather_predictive.humidity AS weather_predictive_humidity, weather_predictive.wind_speed AS weather_predictive_wind_speed, weather_predictive.clouds AS weather_predictive_clouds 
FROM weather_predictive
2024-04-04 11:12:45,244 INFO sqlalchemy.engine.Engine [generated in 0.00134s] {}


In [117]:
# Generate a list of hours for today
current_date = datetime.now()
current_hour = current_date.hour
hours_today = [current_date.replace(
    hour=h, minute=30, second=0, microsecond=0) for h in range(current_hour, 24)]
hourly_df = pd.DataFrame(hours_today, columns=['time_updated'])

df = pd.merge_asof(hourly_df, weather_combined, on='time_updated')

In [105]:
# One hot encode day of the week
weather_predictive_df['weekday'] = weather_predictive_df['time_updated'].dt.day_name()
weather_predictive_df['hour'] = weather_predictive_df['time_updated'].dt.hour
for day in days:
    weather_predictive_df[day] = weather_predictive_df['weekday'] == day

weather_predictive_df['rain'] = weather_predictive_df['type'] == 'Rain'

weather_predictive_df.drop('time_updated', axis=1, inplace=True)
weather_predictive_df.drop('weekday', axis=1, inplace=True)
weather_predictive_df.drop('type', axis=1, inplace=True)

# Get the columns in the right order
weather_predictive_df = weather_predictive_df[['temperature', 'wind_speed', 'humidity', 'hour', 'rain', 'Friday',
            'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday']]


In [107]:
weather_predictive_df

,temperature,wind_speed,humidity,hour,rain,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,9.52,4.31,89,12,False,False,False,False,False,True,False,False
1,9.93,4.83,86,15,True,False,False,False,False,True,False,False
2,9.35,3.20,97,18,True,False,False,False,False,True,False,False
3,9.47,4.46,98,21,True,False,False,False,False,True,False,False
4,11.19,4.01,91,0,True,True,False,False,False,False,False,False
5,10.17,3.20,94,3,True,True,False,False,False,False,False,False
6,9.84,4.47,88,6,False,True,False,False,False,False,False,False
7,11.30,3.96,86,9,True,True,False,False,False,False,False,False
8,14.03,7.27,70,12,True,True,False,False,False,False,False,False
9,12.97,6.57,78,15,True,True,False,False,False,False,False,False


In [108]:
with open(f'../ML_models/station_{station_id}.pkl', 'rb') as file:
    # Load the model from the file
    poly_reg_model = pickle.load(file)

poly = PolynomialFeatures(degree=3, include_bias=False)
poly_features = poly.fit_transform(weather_predictive_df)


In [109]:

weather_predictive_df['predicted_available'] = poly_reg_model.predict(poly_features)
weather_predictive_df['predicted_available'] = np.clip(
    weather_predictive_df['predicted_available'], 0, station_data['bike_stands'])

In [116]:
combined_df = pd.concat(
    [station_data[['hour', 'available_bikes']], weather_predictive_df[['hour', 'predicted_available']]])
combined_df.replace(np.nan, None, inplace=True)
combined_df

,hour,available_bikes,predicted_available
0,0,2.0,None
1,1,2.0,None
2,2,2.0,None
3,3,2.0,None
4,4,3.0,None
5,5,6.833333,None
6,6,11.0,None
7,7,10.0,None
8,8,15.916667,None
9,9,17.833333,None


In [91]:
    data = {'data': []
            }
    for hour, avail_bikes, pred_avail in zip(combined_df['hour'].values.tolist(), combined_df['available_bikes'].values.tolist(), combined_df['predicted_available'].values.tolist()):
        data['data'].append([str(hour), avail_bikes, pred_avail])
    print(data, file=sys.stdout)

,hour,available_bikes,bike_stands
0,0,11.0,20.0
1,1,11.0,20.0
2,2,11.0,20.0
3,3,11.0,20.0
4,4,11.0,20.0


In [92]:
combined_df = pd.concat([station_data[['hour', 'available_bikes']], df[['hour', 'predicted_available']]])
combined_df.replace(np.nan, None, inplace=True)
combined_df


KeyError: "['predicted_available'] not in index"

In [ ]:
data = {'data': []
        }
for hour, avail_bikes, pred_avail in zip(combined_df['hour'].values.tolist(), combined_df['available_bikes'].values.tolist(), combined_df['predicted_available'].values.tolist()):
    print(hour)
    data['data'].append([str(hour), avail_bikes, pred_avail])
print(data, file=sys.stdout)

0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
3
4
4
4
4
4
4
4
4
4
4
4
4
5
5
5
5
5
5
5
5
5
5
5
5
6
6
6
6
6
6
6
6
6
6
6
6
7
7
7
7
7
7
7
7
7
7
7
7
8
8
8
8
8
8
8
8
8
8
8
8
9
9
9
9
9
9
9
9
9
9
9
9
10
10
10
10
10
10
10
10
10
10
10
10
11
11
11
11
11
11
11
11
11
11
11
11
12
12
12
12
12
12
12
12
12
12
12
12
13
13
13
13
13
13
13
13
13
13
13
13
14
14
14
14
14
14
14
14
14
14
14
14
15
15
15
15
15
15
15
15
15
15
15
15
16
16
16
16
16
16
16
16
16
16
16
16
17
17
17
17
17
17
17
17
17
17
17
17
18
18
18
18
18
18
18
18
18
18
18
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
{'data': [['0', 5.0, None], ['0', 5.0, None], ['0', 5.0, None], ['0', 5.0, None], ['0', 5.0, None], ['0', 5.0, None], ['0', 5.0, None], ['0', 5.0, None], ['0', 5.0, None], ['0', 5.0, None], ['0', 5.0, None], ['0', 5.0, None], ['1', 5.0, None], ['1', 5.0, None], ['1', 5.0, None], ['1', 5.0, None], ['1', 5.0, None], ['1', 5.0, None], ['1', 5.0, None], ['1', 5.0, None], ['1', 5

In [ ]:
df

,temperature,wind_speed,humidity,hour,rain,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,predicted_available
0,8.18,5.66,92,0,True,False,False,False,False,False,False,True,0.042450
1,8.21,6.69,91,1,True,False,False,False,False,False,False,True,1.008216
2,8.18,6.69,93,2,True,False,False,False,False,False,False,True,0.359445
3,8.36,6.17,94,3,False,False,False,False,False,False,False,True,8.298418
4,8.46,6.17,93,4,True,False,False,False,False,False,False,True,4.162119
5,8.46,7.72,94,5,True,False,False,False,False,False,False,True,2.920615
6,8.05,6.69,95,6,True,False,False,False,False,False,False,True,3.863218
7,7.50,7.20,93,7,False,False,False,False,False,False,False,True,11.260365
8,7.37,7.20,93,8,False,False,False,False,False,False,False,True,11.951043
9,7.53,7.72,94,9,False,False,False,False,False,False,False,True,12.420441
